## PDF Loader

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
loader = PyPDFLoader("./teeth.pdf")
loader.load()[0]

Document(metadata={'source': './teeth.pdf', 'page': 0}, page_content='周病\n健康照護手冊\n2019 年 1 月\n本項經費係由衛生福利部菸害防制及衛生保健基金支應 衛生福利部委託臺北醫學大學口腔醫學院牙醫學系編印\n衛生福利部\nMinistry of Health and Welfare\n臺北醫學大學口腔醫學院\n牙醫學系\n牙\nࠦࠫɨʹ\x12\x12\x1b\x11\x1a')

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 128)
doc_split = loader.load_and_split(splitter)

## Embedding

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [6]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

/Users/klin/git/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
vectorstore = Chroma.from_documents(
    documents=doc_split,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

## OpenAI client

In [9]:
import os
from dotenv import load_dotenv 

from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

In [10]:
load_dotenv()

In [11]:
client = OpenAI(
    api_key=os.getenv("API_KEY"),  # This is the default and can be omitted
)

/var/folders/ch/lpq22ds90kn4v5gvkqbbmmtw0000gn/T/ipykernel_55598/3028937250.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(


In [12]:
template = """
Question: {question}

Answer:
"""
prompt = PromptTemplate.from_template(template)
pdf_qa = ConversationalRetrievalChain.from_llm(
    llm=client,
    condense_question_prompt=prompt,
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    verbose=False,
)

In [13]:
result = pdf_qa.invoke(
        {"question": "牙周病該怎麼治療? 解釋後並用英文回答", "chat_history": []})
result

{'question': '牙周病該怎麼治療? 解釋後並用英文回答',
 'chat_history': [],
 'answer': '\n��周���的治���可以分���非手��治���和手��治�������。非手��治���包��口���清���及�����、支持性治���和自我�����。手��治�����包����周手��、���������手��和�����手��。治�����程中，定期回�����查和����也是��重要的一���。此外，����也能提高治���的成功率。��周���的治���需要�������人情��和�����建��，�����最��合的治���方式。The treatment for periodontal disease can be divided into non-surgical and surgical treatments. Non-surgical treatments include oral hygiene and maintenance, supportive therapy, and self-care. Surgical treatments include periodontal surgery, root',
 'source_documents': [Document(metadata={'page': 14, 'source': './teeth.pdf'}, page_content='14\n部位，增加治療效果。但文獻指出，目前並沒有確切的資料佐證何種\n雷射對牙周治療最有效益，也沒有確切的研究佐證，使用雷射會有優\n於傳統的牙周治療。所以民眾應在牙醫師的協助之下，選擇對自己最\n適合的治療方法。\n支持性治療\n這是牙周病治療最重要的一環\n24\n。\n病人每三個月到六個月的定期回診，檢查洗牙，並且記錄牙周囊\n袋深度，可以確保治療的結果，並且倘若有復發的跡象，可以經由早\n期發現而早期治療\n24\n。支持性的治療，包括牙醫專業的洗牙與牙周探\n測紀錄。倘若在固定的回診中，發現復發或惡化的牙周囊袋，可以用\n手術或其他方式回復牙周健康。 \nD\t牙周病自我照護\n31,32,33\n牙周病患者一定要遵守吃完飯後就刷牙的原則，且要定期做口腔\n檢查與洗牙。牙周病就像高血壓、心臟病等慢性